<a href="https://colab.research.google.com/github/Xenosians/pemop-week-13/blob/main/Parkinsons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.stats import shapiro, skew, kurtosis
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import probplot
from sklearn.metrics import confusion_matrix

In [ ]:
url = "https://raw.githubusercontent.com/Xenosians/pemop-week-13/main/parkinsons.txt"
data = pd.read_csv(url)

data = data.drop(['name'], axis=1)
X = data.drop(['status'], axis=1)
y = data['status']

In [ ]:
data

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X.apply(shapiro)

In [ ]:
X.apply(skew)

In [ ]:
X.apply(kurtosis)

In [ ]:
def plot(data):
    if isinstance(data, np.ndarray):
        num_columns = data.shape[1]
        column_names = [f'Column {i + 1}' for i in range(num_columns)]
    else:
        num_columns = len(data.columns)
        column_names = data.columns

    fig, axes = plt.subplots(nrows=num_columns, ncols=2, figsize=(10, 2 * num_columns))

    for i in range(num_columns):
        if isinstance(data, np.ndarray):
            column_data = data[:, i]
        else:
            column_data = data.iloc[:, i]

        probplot(column_data, plot=axes[i, 0])
        axes[i, 0].set_title(f'Probability Plot - {column_names[i]}')

        axes[i, 1].hist(column_data, bins=20, density=True, alpha=0.5)
        axes[i, 1].set_title(f'Histogram - {column_names[i]}')

    plt.tight_layout()
    plt.show()

In [ ]:
plot(X)

In [ ]:
X.shape

In [ ]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=15, batch_size=8, validation_split=0.3, callbacks=[checkpoint], verbose=1)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")
conf_mat = confusion_matrix(y_test, y_pred)
print("\nTest Accuracy:", test_acc * 100)

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(conf_mat, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()